# COMP 7150 Assignment 5

Due: April 4, 2023 before midnight.

NAME: _______________

---

**How each problem is graded**

This is a general guideline.  There might be some variant for a specific problem.

* Answering the question in English: 25%
    + Thorough answering the question, explaning the findings, discussing the comparison: 10-25%
    + Not answering the question in English: 0%.
    + Note: An answer is written in English to address a specific question. The answer must come first.  
* Code: 50%
    + Code is correct: 50%
    + Code is correct, but missing some minor elements: 25-40%
    + Incorrect solution, but make an effort: 10%-25%
    + Empty: 0% 
    + Note: code is not an answer. Code shows how you get an answer.  Code comes after an answer.
* Explanation of code: 10%
    + Concise English explanation of solution/code: 10%
    + No explanation: 0%
* Testing: 15%
    + Providing adequate testing of code: 15%
    + Code operational but does not show how the answer(s) were generated: 5%
    + Code not runnable or no testing: 0%


**How to turn in your assignment**

+ Export your notebook to an HTML file.
+ Upload it to the appropriate folder in Assignments on Canvas.

#### Dataset

The dataset you'll used for this assignment is diabetes.csv

The target variable is Outcome. The other 8 variables are features.

This dataset was collected from pregnant women.  Each row corresponds to a person.  An Outcome of 0 means the person is not diagnosed with diabetes.  An Outcome of 1 means the person is diagnosed with diabetes.

**Each problem is worth 20 points.**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt

diabetes = pd.read_csv('./../Sem_2/Diabetes/diabetes.csv')

In [2]:
print(diabetes.shape)
diabetes.head()


(768, 9)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


---

**Problem 1**

Suppose that you want to compare these two methods of cross validation: shuffle and split, and 12-fold cross validation.

With shuffle and split, you will probably want to specify the same number test data points and training data points as 12-fold cross validation.

How many test data points do you have to set in shuffle split to ensure that the two methods have the same number of test data points (and consequently train data points)?



1) From the shape of the dataset there are 768 data points. As per the KFold cross validation with k=12, the entire data is split into 12 equal subsets i.e 12 * 64.
2) Thus we have a train set of 11*64 rows and test set of 64 rows
3) Finally the data must have 704 train data points and 64 test data points for comparing both cross validation methods

In [3]:
# Let us verify this using the KFold
from sklearn.model_selection import KFold

kfold=KFold(n_splits=12,random_state=42,shuffle=True)

for train,test in kfold.split(diabetes):
    print(f'No. of train data points are {len(train)} and no. of test data points are {len(test)}')
    break

No. of train data points are 704 and no. of test data points are 64


---

**Problem 2**

Sklearn's KFold cross validation has a parameter called "shuffle".  If this parameter is set to True, the data is shuffled randomly before the cross validation begins.

Use sklearn's KFold and cross_validate to implement a function that randomly perform 5-Fold cross validation 5 times on given a model and data, as specified below.

In [4]:
#
# Input: model, features (X), target (y)
# Output: average f1 score of the cross validation
#

from sklearn.model_selection import cross_validate

def five_five_fold_cross_validate(model, X, y):
    
    f1_score=[]
    kfold=KFold(n_splits=5,shuffle=True,random_state=86)
    
    for i in range(5):
        cv=cross_validate(model,X,y,cv=kfold)
        #print(cv)
        f1_score.append(cv['test_score'].mean())
    
    return round(sum(f1_score)/len(f1_score),4)

---
**Problem 3**

Validate a 10-nearest neighbor model on the diabetes dataset using the five_five_fold_cross_validate method above.

Use "Outcome" for y, and any two features for X.  Try to get as high scores as you can by choose two good features. 

In [5]:
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.feature_selection import SelectKBest, f_classif

features=diabetes.drop('Outcome',axis=1)
target=diabetes['Outcome']

selector=SelectKBest(f_classif,k=2)
selector.fit(features,target)
cols=selector.get_feature_names_out()

knn=KNN(n_neighbors=10)

print(f'The best score of 10 neighbor KNN model is {five_five_fold_cross_validate(knn,features[cols],target)}')

The best score of 10 neighbor KNN model is 0.7278


---
**Problem 4**

Validate a 10-nearest neighbor model on the diabetes dataset using the five_five_fold_cross_validate method above.

Use "Outcome" for y.

For X, choose the first two principal components obtained by PCA.  To train PCA, use all features of diabetes.

In [6]:
from sklearn.decomposition import PCA

pca=PCA(n_components=2)
X_pca=pca.fit_transform(features)

print(f'The best score of 10 neighbor KNN model with two Principal components is {five_five_fold_cross_validate(knn,X_pca,target)}')

The best score of 10 neighbor KNN model with two Principal components is 0.7135


---
**Problem 5**

Validate a most-frequent dummy classifier on the diabetes dataset using the five_five_fold_cross_validate method above.

Use "Outcome" for y.

For X, choose the first two principal components obtained by PCA.  To train PCA, use all features of diabetes.

In [7]:
from sklearn.dummy import DummyClassifier

dummy=DummyClassifier(strategy='most_frequent')
print(f'The best score of Dummy Classifier with two Principal components is {five_five_fold_cross_validate(dummy,X_pca,target)}')

The best score of Dummy Classifier with two Principal components is 0.651
